In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Merge 2 DF

In [2]:
df = pd.read_csv('hotel_location.csv')
df.head()

,hotel_name_,location,zip_code,city,latitude,longitude
0,RedDoorz near East Avenue Medical Center,"90 Matatag Street, Quezon City, 1100 Manila, P...",1100,Quezon City,14.650991,121.048615
1,Skylab Guesthouse,"213B, Aldrin Street, Moonwalk Village 1, Parañ...",1700,Parañaque,14.499822,120.991487
2,The Excelsior Hotel Las Piñas,"1345 Alabang–Zapote Road, Las Pinas, 1750 Mani...",1750,Las Piñas,14.480424,120.980870
3,Rothman Hotel,"1633 M. Adriatico St., Malate, 1004 Manila, Ph...",1004,Manila,14.594891,120.978262
4,OYO 772 Remedios Apartment,"909, Remedios St, Manila Malate Manila, 1004 M...",1004,Manila,14.594891,120.978262


In [3]:
df1 = pd.read_csv('hotel_surroundings.csv')
df1.head()

,hotel_name_,nearest_airport,nearest_resto,nearest_train,nearest_ocean,nearest_other,number_close
0,RedDoorz near East Avenue Medical Center,15.0,0.10,12.0,12.0,0.90,21
1,Skylab Guesthouse,12.0,0.20,11.0,6.0,0.95,21
2,The Excelsior Hotel Las Piñas,8.0,0.04,17.0,11.0,0.55,20
3,Rothman Hotel,8.0,0.10,0.9,0.3,0.25,19
4,OYO 772 Remedios Apartment,7.0,0.10,0.5,1.0,0.60,19


In [4]:
df_merge = pd.merge(df, df1,  how='left', on='hotel_name_')
df_merge.head()

,hotel_name_,location,zip_code,city,latitude,longitude,nearest_airport,nearest_resto,nearest_train,nearest_ocean,nearest_other,number_close
0,RedDoorz near East Avenue Medical Center,"90 Matatag Street, Quezon City, 1100 Manila, P...",1100,Quezon City,14.650991,121.048615,15.0,0.10,12.0,12.0,0.90,21
1,Skylab Guesthouse,"213B, Aldrin Street, Moonwalk Village 1, Parañ...",1700,Parañaque,14.499822,120.991487,12.0,0.20,11.0,6.0,0.95,21
2,The Excelsior Hotel Las Piñas,"1345 Alabang–Zapote Road, Las Pinas, 1750 Mani...",1750,Las Piñas,14.480424,120.980870,8.0,0.04,17.0,11.0,0.55,20
3,Rothman Hotel,"1633 M. Adriatico St., Malate, 1004 Manila, Ph...",1004,Manila,14.594891,120.978262,8.0,0.10,0.9,0.3,0.25,19
4,OYO 772 Remedios Apartment,"909, Remedios St, Manila Malate Manila, 1004 M...",1004,Manila,14.594891,120.978262,7.0,0.10,0.5,1.0,0.60,19


## Group by and Agg

In [38]:
def f(x):
    d = {}
    d['hotel_count'] = x['city'].count()
    d['mean_estab'] = round(x['number_close'].mean(), 2)
    return pd.Series(d, index=['hotel_count', 'mean_estab'])

In [39]:
df_final = df_merge.groupby('city').apply(f).reset_index()
df_final

,city,hotel_count,mean_estab
0,Las Piñas,6.0,19.33
1,Makati City,77.0,20.23
2,Mandaluyong,8.0,21.00
3,Manila,77.0,19.55
4,Marikina,1.0,21.00
5,Muntinlupa,7.0,17.57
6,Navotas,4.0,17.00
7,Parañaque,27.0,20.81
8,Pasay City,31.0,20.77
9,Pasig City,8.0,21.00


In [83]:
# df_hotel_counts = df_merge.groupby(by = ['city', 'latitude', 'longitude'])['city'].count().reset_index(name = 'hotel_counts')

## Mapping using Plotly

In [7]:
import json

In [23]:
ncr_cities = json.load(open('MetropolitantManila.json', 'r'))

In [24]:
# for i in range(0,17):
#     print(ncr_cities['features'][i]['properties']['NAME_2'])
# ncr_cities['features'][13]['properties']

In [40]:
cities_map = {}
for feature in ncr_cities['features']:
    feature['id'] = feature['properties']['ID_2']
    cities_map[feature['properties']['NAME_2']] = feature['id']

In [26]:
cities_map

{'Kalookan City': 950,
 'Las Piñas': 951,
 'Makati City': 952,
 'Malabon': 953,
 'Mandaluyong': 954,
 'Manila': 955,
 'Marikina': 956,
 'Muntinlupa': 957,
 'Navotas': 958,
 'Parañaque': 959,
 'Pasay City': 960,
 'Pasig City': 961,
 'Pateros': 962,
 'Quezon City': 963,
 'San Juan': 964,
 'Taguig': 965,
 'Valenzuela': 966}

In [41]:
df_final['id'] = df_final['city'].apply(lambda x: cities_map[x])

In [43]:
df_final

,city,hotel_count,mean_estab,id
0,Las Piñas,6.0,19.33,951
1,Makati City,77.0,20.23,952
2,Mandaluyong,8.0,21.00,954
3,Manila,77.0,19.55,955
4,Marikina,1.0,21.00,956
5,Muntinlupa,7.0,17.57,957
6,Navotas,4.0,17.00,958
7,Parañaque,27.0,20.81,959
8,Pasay City,31.0,20.77,960
9,Pasig City,8.0,21.00,961


In [16]:
import plotly.express as px

In [17]:
import plotly.io as pio
pio.renderers.default = 'chrome'

In [44]:
fig = px.choropleth(df_final, 
                    locations = 'id', 
                    geojson = ncr_cities, 
                    color = 'hotel_count',
                    hover_name = 'city',
                   hover_data = ['mean_estab'])
fig.update_geos(fitbounds = 'locations', visible = False)
fig.show()